In [3]:
from tabpfn import TabPFNRegressor

In [137]:
from abc import abstractmethod
from tabicl.prior.dataset import SCMPrior
from tabicl.prior.prior_config import DEFAULT_FIXED_HP, DEFAULT_SAMPLED_HP

import numpy as np
from scipy.stats import loguniform
import joblib
import math

import torch
import torch.nn.functional as F
from torch import Tensor
from torch.nested import nested_tensor
from torch.utils.data import IterableDataset

from typing import Optional, Tuple, Dict, Any, Union

from tabicl.prior.dataset import DummyPrior
    
mcar_prior = MCARPrior(num_missing=10, max_features = 20, max_seq_len=100)

# X, _, _ = mcar_prior.generate_dataset({
#     'prior_type': 'mlp_scm',
#     'num_features': 10,
#     'num_classes': 2,
#     'device': 'cpu',
#     'seq_len': 20,
#     'train_size': 10,
#     'max_features': 20,
#     'min_features': 5,
#     'max_classes': 10,
#     'cat_prob': 1.0,
#     'scale_by_max_features': True,
#     'multiclass_type': 'rank',
#     'multiclass_ordered_prob': 0.2
# })

X, y, d, seq_lens, train_sizes = mcar_prior.get_batch(batch_size = 4)

print(X.shape)
print(seq_lens, train_sizes)

# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.heatmap(torch.isnan(X), cbar=False, cmap='Blues')

torch.Size([4, 2000, 118])
tensor([2000., 2000., 2000., 2000.]) tensor([1990., 1990., 1990., 1990.])


In [141]:
from tabpfn.model.bar_distribution import FullSupportBarDistribution, get_bucket_limits

In [143]:
y[0]

tensor([ 0.7718, -0.4635,  0.2764,  ..., -2.1629, -0.4673, -0.3455])

In [150]:
get_bucket_limits(num_outputs = 100, ys = y[0])

tensor([-3.8737e+00, -2.4309e+00, -2.0873e+00, -1.9186e+00, -1.7537e+00,
        -1.6525e+00, -1.5780e+00, -1.4858e+00, -1.3852e+00, -1.3385e+00,
        -1.2987e+00, -1.2466e+00, -1.2169e+00, -1.1501e+00, -1.0999e+00,
        -1.0378e+00, -1.0113e+00, -9.6897e-01, -9.2145e-01, -8.8394e-01,
        -8.4370e-01, -8.0649e-01, -7.7390e-01, -7.4602e-01, -7.1809e-01,
        -6.7251e-01, -6.4145e-01, -6.1843e-01, -5.9119e-01, -5.6030e-01,
        -5.3457e-01, -5.0215e-01, -4.7838e-01, -4.5791e-01, -4.3009e-01,
        -4.1545e-01, -3.8019e-01, -3.5528e-01, -3.3076e-01, -3.0420e-01,
        -2.7439e-01, -2.4172e-01, -2.2195e-01, -1.9647e-01, -1.7418e-01,
        -1.4726e-01, -1.2405e-01, -9.5777e-02, -7.5509e-02, -5.1924e-02,
        -3.1530e-02, -3.8033e-03,  1.9635e-02,  4.3842e-02,  7.4794e-02,
         9.6513e-02,  1.2021e-01,  1.6955e-01,  1.9146e-01,  2.2836e-01,
         2.6040e-01,  2.8865e-01,  3.1268e-01,  3.3820e-01,  3.6411e-01,
         3.8537e-01,  4.1810e-01,  4.4873e-01,  4.7

In [ ]:
# Now we just need to train on this data

In [151]:
import torch

batch1 = torch.load('/Users/jfeit/tabular/mcpfn/prior/batch_000000.pt')

In [154]:
batch1['X'].isnan().sum()

tensor(760)

In [48]:
batch1['X']

torch.Size([26227712])

In [46]:
batch1['y'].shape

torch.Size([512, 1024])

In [43]:
batch1['d']

tensor([ 13,  13,  13,  13,  36,  36,  36,  36,  37,  37,  37,  37,  47,  47,
         47,  47,  42,  42,  42,  42,  97,  97,  97,  97,  37,  37,  37,  37,
         75,  75,  75,  75,  43,  43,  43,  43,  70,  70,  70,  70,  73,  74,
         74,  74,  55,  55,  55,  55,  46,  46,  46,  46,  94,  94,  94,  94,
         18,  18,  18,  18,  60,  60,  60,  60,  44,  44,  44,  44,  83,  83,
         83,  83,  47,  47,  47,  47,  36,  36,  36,  36,  20,  20,  20,  20,
         30,  30,  30,  30,  43,  43,  43,  43,  61,  61,  61,  61,  33,  33,
         33,  33,   5,   5,   5,   5, 100, 100, 100, 100,  39,  39,  39,  39,
          4,   4,   4,   4,  79,  79,  79,  79,  97,  97,  97,  97,  80,  80,
         80,  80,  62,  62,  62,  62,  10,  10,  10,  10,  52,  52,  52,  52,
         82,  82,  82,  82,   8,   8,   8,   8,  58,  58,  58,  58,  78,  78,
         78,  78,  12,  12,  12,  12,  72,  72,  72,  72,  80,  80,  80,  80,
         97,  99,  99,  99,  29,  29,  29,  29,  55,  55,  55,  

In [5]:
# Load diabetes dataset
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

# Split into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
model = TabPFNRegressor(device="cpu")

model.fit(X_train, y_train)

/Users/jfeit/tabular/.venv/lib/python3.10/site-packages/tabpfn/regressor.py:460: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  check_cpu_warning(


TabPFNRegressor(device='cpu')

In [8]:
model.predict(X_test)

array([151.11984 , 181.23395 , 152.39328 , 270.66843 , 116.048615,
        98.57599 , 280.28693 , 204.01453 ,  92.98259 , 104.42664 ,
        97.20478 , 156.68275 ,  79.03818 , 221.29614 ,  95.34686 ,
       113.29086 , 232.82916 , 263.02063 , 202.36508 , 226.96875 ,
       214.99869 ,  90.550705,  76.045364, 204.91766 , 148.44948 ,
       159.32785 , 191.49872 , 179.13577 ,  71.97873 , 108.86607 ,
       166.0928  ,  90.06023 , 127.83388 , 188.93164 , 168.59038 ,
       215.19928 , 119.307785, 116.949844, 145.44827 ,  73.5537  ,
        77.18077 ,  99.99011 , 167.50687 , 147.84912 , 172.1824  ,
        76.06234 ,  81.00776 , 105.166275,  74.43037 , 161.98457 ,
       140.27203 ,  80.37433 , 104.437836, 101.16085 , 178.6712  ,
       142.64389 ,  89.98228 , 221.62193 , 118.673615,  80.18475 ,
       192.11647 , 208.27834 , 117.20884 ,  95.30004 , 119.84745 ,
       217.77151 , 175.74219 , 167.14984 , 120.115524, 143.5263  ,
       187.05565 , 203.7262  , 228.974   , 142.24925 ,  84.665

In [12]:
model.model_

PerFeatureTransformer(
  (encoder): SequentialEncoder(
    (0): RemoveEmptyFeaturesEncoderStep()
    (1): NanHandlingEncoderStep()
    (2): VariableNumFeaturesEncoderStep()
    (3): InputNormalizationEncoderStep()
    (4): VariableNumFeaturesEncoderStep()
    (5): LinearInputEncoderStep(
      (layer): Linear(in_features=4, out_features=192, bias=False)
    )
  )
  (y_encoder): SequentialEncoder(
    (0): NanHandlingEncoderStep()
    (1): LinearInputEncoderStep(
      (layer): Linear(in_features=2, out_features=192, bias=True)
    )
  )
  (transformer_encoder): LayerStack(
    (layers): ModuleList(
      (0-11): 12 x PerFeatureEncoderLayer(
        (self_attn_between_features): MultiHeadAttention()
        (self_attn_between_items): MultiHeadAttention()
        (mlp): MLP(
          (linear1): Linear(in_features=192, out_features=768, bias=False)
          (linear2): Linear(in_features=768, out_features=192, bias=False)
        )
        (layer_norms): ModuleList(
          (0-2): 3 x 

In [29]:
import numpy as np

def create_train_test_sets(X):
    # Get missing indices in X
    missing_indices = np.where(np.isnan(X))
    
    non_missing_indices = np.where(~np.isnan(X))
    
    train_X = np.zeros((len(non_missing_indices[0]), X.shape[0] + X.shape[1] - 2))
    train_y = np.zeros(len(non_missing_indices[0]))
    test_X = np.zeros((len(missing_indices[0]), X.shape[0] + X.shape[1] - 2))
    test_y = np.zeros(len(missing_indices[0]))
    
    for k, (i,j) in enumerate(zip(non_missing_indices[0], non_missing_indices[1])):
        # Get row without j-th column
        row = np.delete(X[i,:], j)
        # Get column without i-th row
        col = np.delete(X[:,j], i)
        
        # Create train set
        train_X[k,:] = np.concatenate((row, col))
        train_y[k] = X[i,j]
        
    
    for k, (i,j) in enumerate(zip(missing_indices[0], missing_indices[1])):
        # Get row without j-th column
        row = np.delete(X[i,:], j)
        # Get column without i-th row
        col = np.delete(X[:,j], i)
        
        # Create train set
        test_X[k,:] = np.concatenate((row, col))
        test_y[k] = np.nan
        
    return train_X, train_y, test_X, test_y
    
X = np.array([[1, 3, 5], [7, np.nan, 6], [-1, 2, np.nan]])
train_X, train_y, test_X, test_y = create_train_test_sets(X)

print(train_X)
print(train_y)
print(test_X)
print(test_y)

[[ 3.  5.  7. -1.]
 [ 1.  5. nan  2.]
 [ 1.  3.  6. nan]
 [nan  6.  1. -1.]
 [ 7. nan  5. nan]
 [ 2. nan  1.  7.]
 [-1. nan  3. nan]]
[ 1.  3.  5.  7.  6. -1.  2.]
[[ 7.  6.  3.  2.]
 [-1.  2.  5.  6.]]
[nan nan]


In [19]:
import torch

In [20]:
X_tnsr = torch.from_numpy(X)

tensor([[ 1.,  3.,  5.],
        [ 7., nan,  6.],
        [-1.,  2., nan]], dtype=torch.float64)